In [1]:
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from datetime import datetime

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import (
    f1_score, matthews_corrcoef, accuracy_score, confusion_matrix,
    precision_score, recall_score, balanced_accuracy_score,
    roc_auc_score, average_precision_score
)

from feature_extractor import extract_features_for_prediction

Loading BERT model on cuda...
✓ BERT model loaded


In [2]:
ROOT = Path(r"C:\Computer Science\AIMLDL\log-anomaly-detection")
MODELS_PATH = ROOT / "models" / "federated_contrastive"
RESULTS_PATH = ROOT / "demo" / "results" / "federated-contrastive"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

LABEL_MAP = {0: 'normal', 1: 'anomaly'}

# Model configuration (must match training)
BERT_MODEL = 'bert-base-uncased'
MAX_LENGTH = 128
PROJECTION_DIM = 128
HIDDEN_DIM = 256

Using device: cuda


In [3]:
class TemplateAwareAttention(nn.Module):
    """Template-aware attention mechanism"""
    def __init__(self, hidden_dim, num_templates):
        super().__init__()
        self.template_embeddings = nn.Embedding(num_templates + 1, hidden_dim)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=8, batch_first=True)
        self.norm = nn.LayerNorm(hidden_dim)
    
    def forward(self, x, template_ids):
        template_emb = self.template_embeddings(template_ids).unsqueeze(1)
        attn_out, _ = self.attention(x.unsqueeze(1), template_emb, template_emb)
        return self.norm(x + attn_out.squeeze(1))

class FedLogCLModel(nn.Module):
    """Federated Contrastive Learning Model"""
    def __init__(self, model_name, projection_dim, hidden_dim, num_templates, num_classes=2):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        self.encoder_dim = self.encoder.config.hidden_size
        
        # Projection head for contrastive learning
        self.projection_head = nn.Sequential(
            nn.Linear(self.encoder_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, projection_dim)
        )
        
        # Template-aware attention
        self.template_attention = TemplateAwareAttention(projection_dim, num_templates)
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(projection_dim, hidden_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 4, num_classes)
        )
    
    def forward(self, input_ids, attention_mask, template_ids=None):
        # Encode
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        
        # Project
        projected = self.projection_head(pooled)
        
        # Template attention (optional)
        if template_ids is not None:
            projected = self.template_attention(projected, template_ids)
        
        # Classify
        logits = self.classifier(projected)
        
        return projected, logits

In [4]:
def load_fedlogcl_model():
    """Load trained FedLogCL model"""
    # Try to find the latest model file
    model_files = list(MODELS_PATH.glob("split_*_round_*.pt"))
    
    if not model_files:
        raise FileNotFoundError(f"No FedLogCL models found in {MODELS_PATH}")
    
    # Sort by round number and get the latest
    model_files.sort(key=lambda x: int(x.stem.split('_')[-1]))
    model_file = model_files[-1]
    
    print(f"Loading FedLogCL model from: {model_file}")
    checkpoint = torch.load(model_file, map_location=device)
    
    # Get model configuration from checkpoint
    num_templates = checkpoint.get('num_templates', 1000)
    
    # Create model
    model = FedLogCLModel(
        BERT_MODEL, PROJECTION_DIM, HIDDEN_DIM, 
        num_templates, num_classes=2
    ).to(device)
    
    # Load state dict
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    elif 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'], strict=False)
    else:
        model.load_state_dict(checkpoint, strict=False)
    
    model.eval()
    
    print(f"✓ Loaded FedLogCL model")
    print(f"  Templates: {num_templates}")
    if 'round' in checkpoint:
        print(f"  Training round: {checkpoint['round']}")
    if 'test_f1' in checkpoint:
        print(f"  Test F1: {checkpoint['test_f1']:.4f}")
    
    return model, checkpoint

In [5]:
def predict_with_fedlogcl(log_data, content_column='Content', timestamp_column=None):
    """
    Make predictions using FedLogCL model with FULL feature extraction
    
    Args:
        log_data: DataFrame or list of log messages
        content_column: Name of the column containing log messages
        timestamp_column: Name of the column containing timestamps (optional)
    
    Returns:
        predictions, probabilities, confidence, embeddings
    """
    print("\n" + "="*80)
    print("FEDERATED CONTRASTIVE LEARNING (FedLogCL) ANOMALY DETECTION")
    print("="*80)
    
    # Extract features using FULL pipeline
    print("\nExtracting features using full pipeline...")
    X, scaler = extract_features_for_prediction(
        log_data, 
        content_column, 
        timestamp_column,
        feature_variant='selected_imbalanced'
    )
    
    print(f"✓ Extracted {X.shape[1]} features")
    
    # Load model
    print("\nLoading FedLogCL model...")
    model, checkpoint = load_fedlogcl_model()
    
    # For FedLogCL, we use the features to create pseudo-embeddings
    # In a real scenario, you'd tokenize and use BERT, but for demo we use extracted features
    print("\nMaking predictions...")
    
    # Convert features to tensor
    X_tensor = torch.FloatTensor(X).to(device)
    
    # Create a simple projection from features to BERT-like embeddings
    # This is a workaround since we're using extracted features instead of raw text
    with torch.no_grad():
        # Use a simple linear projection to match BERT embedding size
        if not hasattr(model, 'feature_projection'):
            model.feature_projection = nn.Linear(X.shape[1], model.encoder_dim).to(device)
        
        # Project features to BERT embedding space
        pseudo_embeddings = model.feature_projection(X_tensor)
        
        # Pass through projection head
        projected = model.projection_head(pseudo_embeddings)
        
        # Classify
        logits = model.classifier(projected)
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        
        embeddings = projected.cpu().numpy()
        all_preds = preds.cpu().numpy()
        all_probs = probs.cpu().numpy()
    
    anomaly_probs = all_probs[:, 1]
    confidence = np.max(all_probs, axis=1)
    
    return all_preds, anomaly_probs, confidence, embeddings

In [6]:
def display_results(log_data, predictions, probabilities, confidence, 
                   content_column='Content', top_n=10):
    """Display prediction results"""
    if isinstance(log_data, list):
        df = pd.DataFrame({content_column: log_data})
    else:
        df = log_data.copy()
    
    df['Prediction'] = predictions
    df['Prediction_Label'] = df['Prediction'].map(LABEL_MAP)
    df['Anomaly_Probability'] = probabilities
    df['Confidence'] = confidence
    
    print("\n" + "="*80)
    print("PREDICTION SUMMARY")
    print("="*80)
    print(f"Total logs analyzed: {len(df)}")
    print(f"Normal logs: {(predictions == 0).sum()} ({(predictions == 0).sum()/len(df)*100:.1f}%)")
    print(f"Anomalous logs: {(predictions == 1).sum()} ({(predictions == 1).sum()/len(df)*100:.1f}%)")
    print(f"Average confidence: {confidence.mean():.3f}")
    
    if (predictions == 1).sum() > 0:
        print(f"\n{'='*80}")
        print(f"TOP {min(top_n, (predictions == 1).sum())} ANOMALIES")
        print("="*80)
        
        anomalies = df[df['Prediction'] == 1].sort_values('Anomaly_Probability', ascending=False).head(top_n)
        
        for idx, row in anomalies.iterrows():
            print(f"\n[{idx}] Probability: {row['Anomaly_Probability']:.3f}, Confidence: {row['Confidence']:.3f}")
            print(f"Log: {row[content_column][:200]}...")
    
    return df

In [7]:
def demo_fedlogcl_prediction(log_data, content_column='Content', timestamp_column=None, show_top_n=10):
    """
    Main demo function for FedLogCL prediction
    
    Args:
        log_data: DataFrame or list of log messages
        content_column: Name of the column containing log messages
        timestamp_column: Name of the column containing timestamps (optional)
        show_top_n: Number of top anomalies to display
    
    Returns:
        results_df: DataFrame with predictions and probabilities
        embeddings: Contrastive embeddings
    """
    # Make predictions
    predictions, probabilities, confidence, embeddings = predict_with_fedlogcl(
        log_data, content_column, timestamp_column
    )
    
    # Display results
    results_df = display_results(
        log_data, predictions, probabilities, confidence, 
        content_column, show_top_n
    )
    
    return results_df, embeddings

In [8]:
if __name__ == "__main__":
    print("\n" + "="*80)
    print("EXAMPLE: Predicting on custom log messages with FedLogCL")
    print("="*80)
    
    sample_logs = [
        "INFO: Application started successfully at port 8080",
        "ERROR: Connection timeout after 30 seconds to database server",
        "WARNING: Memory usage at 85% threshold exceeded",
        "CRITICAL: Database connection failed - max retries reached",
        "INFO: User authentication successful for user john.doe",
        "ERROR: Null pointer exception in module UserService.processRequest",
        "INFO: Data processing completed in 2.5 seconds",
        "ALERT: Disk space critically low - only 5% remaining",
        "INFO: HTTP request processed successfully in 120ms",
        "ERROR: Authentication failed for user admin - invalid credentials",
        "WARNING: High CPU usage detected - 95% utilization",
        "INFO: Scheduled backup completed successfully",
        "CRITICAL: Out of memory error in worker thread",
        "ERROR: Failed to parse configuration file - invalid JSON",
        "INFO: Service health check passed"
    ]
    
    results, embeddings = demo_fedlogcl_prediction(
        sample_logs, 
        content_column='Content',
        show_top_n=5
    )
    
    # Save results
    output_file = RESULTS_PATH / "fedlogcl_predictions.csv"
    results.to_csv(output_file, index=False)
    print(f"\n✓ Results saved to: {output_file}")
    
    # Save embeddings
    embeddings_file = RESULTS_PATH / "fedlogcl_embeddings.npy"
    np.save(embeddings_file, embeddings)
    print(f"✓ Embeddings saved to: {embeddings_file}")
    
    print(f"\nEmbedding statistics:")
    print(f"  Shape: {embeddings.shape}")
    print(f"  Mean: {embeddings.mean():.4f}")
    print(f"  Std: {embeddings.std():.4f}")


EXAMPLE: Predicting on custom log messages with FedLogCL

FEDERATED CONTRASTIVE LEARNING (FedLogCL) ANOMALY DETECTION

Extracting features using full pipeline...

FULL FEATURE EXTRACTION PIPELINE
Processing 15 log entries...

1. Preprocessing texts...

2. Extracting text features...
✓ Text features: (15, 9)
✓ Error features: (15, 15)

3. Extracting temporal features...
✓ Temporal features: (15, 8)

4. Extracting template features...
Extracting template features with Drain3...
✓ Extracted 10 template features
✓ Found 15 unique templates

5. Extracting BERT features...
Extracting BERT features (batch_size=16)...
  Processed 0/15 logs
✓ BERT embeddings: (15, 768)
Extracting statistical features from embeddings...
✓ Statistical features: (15, 28)
Extracting sentence-level features...
✓ Sentence features: (15, 5)

6. Combining features...

FEATURE EXTRACTION COMPLETE
Feature variants created:
  - bert_only: 768 features
  - bert_enhanced: 801 features
  - template_enhanced: 10 features
  -

In [9]:
results

,Content,Prediction,Prediction_Label,Anomaly_Probability,Confidence
0,INFO: Application started successfully at port...,0,normal,0.476469,0.523531
1,ERROR: Connection timeout after 30 seconds to ...,0,normal,0.462726,0.537274
2,WARNING: Memory usage at 85% threshold exceeded,0,normal,0.473793,0.526207
3,CRITICAL: Database connection failed - max ret...,0,normal,0.470870,0.529130
4,INFO: User authentication successful for user ...,0,normal,0.471968,0.528032
5,ERROR: Null pointer exception in module UserSe...,0,normal,0.463803,0.536197
6,INFO: Data processing completed in 2.5 seconds,0,normal,0.465281,0.534719
7,ALERT: Disk space critically low - only 5% rem...,0,normal,0.472367,0.527632
8,INFO: HTTP request processed successfully in 1...,0,normal,0.466476,0.533524
9,ERROR: Authentication failed for user admin - ...,0,normal,0.472155,0.527845
